In [1]:
import os
import re
from collections import defaultdict
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#result_dir = '/result_opt'
#csv_file = 'res.csv'

#result_dir = '/result_full_sys'
#csv_file = 'res_full_sys.csv'

#result_dir = '/result_gnr_full_sys_sweep'
#csv_file = 'res_full_sys_sweep.csv'

#result_dir = '/result_gnr_new_models_sweep'
#csv_file = 'res_new_models_sweep.csv'

#result_dir = '/result_gnr_full_sys_all_models_iter1'
#csv_file = 'result_gnr_full_sys_all_models_iter1.csv'

#result_dir = '/result_spr_all_models_sweep'
#csv_file = 'result_spr_all_models_sweep.csv'

#result_dir = '/result_v1_gnr_full_sys_sweep'
#csv_file = 'result_v1_gnr_full_sys_sweep.csv'

result_dir = '/result_v1_spr_all_models_sweep'
csv_file = 'result_v1_spr_full_sys_sweep.csv'
small_csv = 'result_v1_spr_full_sys_sweep_small.csv'

In [3]:
def get_json(fn):
    j = None
    try:
        with open(fn, 'r') as f:
            j = json.load(f)
    except Exception as e:
        print(f"Opening file {fn} failed with exception {e}")
        #sys.exit(1)
    return j

def get_result_files(directory):
    all_files = defaultdict(lambda: defaultdict(list))
    for root, dirs, files in os.walk(directory):
        for file in files:
            #all_files.append(os.path.join(root, file))
            r = root[len(directory)+1:].split('/')
            all_files[r[0]][r[1]].append(os.path.join(root, file))            
            #all_files[(r[0], r[1])].append(file)
    return all_files

def get_highest_concurrency_file(files):
    t = [(int(f[f.rfind('C')+1:f.rfind('.')]), f) for f in files]
    return max(t)

def extract_results_from_json(fn):
    j = get_json(fn)
    return {
        'Model': j['model_id'],
        'Input Tokens': int(j['total_input_tokens']/j['num_prompts']),
        'Output Tokens': int(j['total_output_tokens']/j['num_prompts']),
        'Concurrency': int(j['request_rate']),
        'Output tput (tokens/sec)': round(j['output_throughput'], 2),
        'Total tput (tokens/sec)': round(j['total_token_throughput'], 2),
        'TTFT avg (ms)': round(j['mean_ttft_ms'], 2),
        'TTFT P50 (ms)': round(j['p50_ttft_ms'], 2),
        'TTFT P90 (ms)': round(j['p90_ttft_ms'], 2),
        'TTFT P99 (ms)': round(j['p99_ttft_ms'], 2),
        'TPOT avg (ms)': round(j['mean_tpot_ms'], 2),
        'TPOT P50 (ms)': round(j['p50_tpot_ms'], 2),
        'TPOT P90 (ms)': round(j['p90_tpot_ms'], 2),
        'TPOT P99 (ms)': round(j['p99_tpot_ms'], 2),
        'Request Throughput': round(j['request_throughput'], 2),
        'Request Latency avg (s)': round(j['mean_e2el_ms']/1000, 2),
        'Request Latency P50 (s)': round(j['p50_e2el_ms']/1000, 2),
        'Request Latency P90 (s)': round(j['p90_e2el_ms']/1000, 2),
        'Request Latency P99 (s)': round(j['p99_e2el_ms']/1000, 2),
    }

In [4]:
files = get_result_files(result_dir)

In [5]:
values = []
for batches in files.values():
    for fns in batches.values():
        #print(fns)
        _, fn = get_highest_concurrency_file(fns)
        #print(fn, "***************************************************")
        values.append(extract_results_from_json(fn))
df = pd.DataFrame(values)
df = df.sort_values(by=['Model', 'Input Tokens'], ascending=[True, True])
df

,Model,Input Tokens,Output Tokens,Concurrency,Output tput (tokens/sec),Total tput (tokens/sec),TTFT avg (ms),TTFT P50 (ms),TTFT P90 (ms),TTFT P99 (ms),TPOT avg (ms),TPOT P50 (ms),TPOT P90 (ms),TPOT P99 (ms),Request Throughput,Request Latency avg (s),Request Latency P50 (s),Request Latency P90 (s),Request Latency P99 (s)
15,Qwen/Qwen2.5-1.5B-Instruct,128,1024,320,3226.78,3630.13,482.74,297.47,994.03,2863.57,97.42,97.88,103.83,105.58,3.15,100.14,100.40,106.77,108.27
16,Qwen/Qwen2.5-1.5B-Instruct,1024,1024,192,1714.79,3429.58,1576.82,452.17,3997.20,16967.97,107.02,103.03,125.74,141.85,1.67,111.05,110.64,131.66,145.68
17,Qwen/Qwen2.5-1.5B-Instruct,2048,1024,96,1032.49,3097.47,1817.97,1213.89,4460.52,10516.76,90.58,90.29,101.40,104.59,1.01,94.48,94.48,104.93,108.68
18,Qwen/Qwen2.5-7B-Instruct,128,1024,16,151.80,170.78,279.05,249.45,400.99,497.10,103.81,103.78,110.47,111.10,0.15,106.48,106.49,113.26,113.90
19,Qwen/Qwen2.5-7B-Instruct,1024,1024,8,78.36,156.72,1059.02,1127.87,2398.48,4026.97,99.80,99.02,105.13,107.80,0.08,103.15,101.96,108.69,111.00
20,Qwen/Qwen2.5-7B-Instruct,2048,1024,8,73.29,219.86,1440.87,1450.79,3206.32,4835.92,107.01,107.41,110.54,114.30,0.07,110.91,110.72,114.72,117.66
33,deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B,128,1024,320,3369.07,3790.20,379.65,256.65,754.33,1922.82,90.46,85.38,111.55,115.84,3.29,92.92,88.23,114.44,118.78
34,deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B,1024,1024,192,1662.24,3324.49,1631.80,474.67,4200.35,17260.30,110.62,109.31,130.77,134.16,1.62,114.80,112.28,134.71,137.83
35,deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B,2048,1024,96,1025.01,3075.04,1609.82,1045.83,2323.97,10532.51,89.22,89.10,111.71,118.55,1.00,92.88,92.05,115.59,122.08
36,deepseek-ai/DeepSeek-R1-Distill-Qwen-7B,128,1024,32,297.90,335.14,309.00,308.19,476.27,643.92,102.51,101.97,111.57,113.21,0.29,105.18,104.80,114.47,116.29


In [6]:
df.to_csv(csv_file, index=False)
df[['Model', 'Input Tokens', 'Output Tokens', 'Concurrency', 'Output tput (tokens/sec)', 'TTFT P90 (ms)', 'TPOT P90 (ms)']].to_csv(small_csv, index=False)

In [7]:
df[['Model', 'Input Tokens', 'Output Tokens', 'Concurrency', 'Output tput (tokens/sec)', 'TTFT P90 (ms)', 'TPOT P90 (ms)']]

,Model,Input Tokens,Output Tokens,Concurrency,Output tput (tokens/sec),TTFT P90 (ms),TPOT P90 (ms)
15,Qwen/Qwen2.5-1.5B-Instruct,128,1024,320,3226.78,994.03,103.83
16,Qwen/Qwen2.5-1.5B-Instruct,1024,1024,192,1714.79,3997.20,125.74
17,Qwen/Qwen2.5-1.5B-Instruct,2048,1024,96,1032.49,4460.52,101.40
18,Qwen/Qwen2.5-7B-Instruct,128,1024,16,151.80,400.99,110.47
19,Qwen/Qwen2.5-7B-Instruct,1024,1024,8,78.36,2398.48,105.13
20,Qwen/Qwen2.5-7B-Instruct,2048,1024,8,73.29,3206.32,110.54
33,deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B,128,1024,320,3369.07,754.33,111.55
34,deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B,1024,1024,192,1662.24,4200.35,130.77
35,deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B,2048,1024,96,1025.01,2323.97,111.71
36,deepseek-ai/DeepSeek-R1-Distill-Qwen-7B,128,1024,32,297.90,476.27,111.57
